<a href="https://colab.research.google.com/github/Shibu4064/EMNLP/blob/main/fine_tuning_for_bangla_hate_speech.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q transformers
!pip install -q datasets
!pip install transformers[torch]
!pip install transformers[tf-cpu]
!pip install transformers[flax]
# !pip install accelerate -U

import multiprocessing
import pandas as pd
import numpy as np
import torch
import matplotlib.pyplot as plt
import transformers

from sklearn.model_selection import train_test_split
from datasets import Dataset
from transformers import AutoModelForMaskedLM
from transformers import AutoTokenizer, AutoConfig
from transformers import BertForMaskedLM, DistilBertForMaskedLM
from transformers import BertTokenizer, DistilBertTokenizer
from transformers import RobertaTokenizer, RobertaForMaskedLM
from transformers import Trainer, TrainingArguments
from transformers import DataCollatorForLanguageModeling
from tokenizers import BertWordPieceTokenizer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 61.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 32.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 115.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 82.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.6/225.6 MB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.8/83.8 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 451.2/451.2 kB 40.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
SEED_SPLIT = 0
SEED_TRAIN = 0

MAX_SEQ_LEN = 128
TRAIN_BATCH_SIZE = 16
EVAL_BATCH_SIZE = 16
LEARNING_RATE = 2e-5
LR_WARMUP_STEPS = 100
WEIGHT_DECAY = 0.01

In [3]:
import gdown

dataset_link = "https://drive.google.com/uc?id=1UEzbztTQUBgKbql0U3QHxam7u1YkEGoZ"

gdown.download(dataset_link,"dataset.csv")



Downloading...
From: https://drive.google.com/uc?id=1UEzbztTQUBgKbql0U3QHxam7u1YkEGoZ
To: /content/dataset.csv
100%|██████████| 30.9M/30.9M [00:00<00:00, 203MB/s]


'dataset.csv'

In [4]:
dtf_mlm = pd.read_csv('/content/dataset.csv')
dtf_mlm = dtf_mlm.rename(columns={"review_content": "text"})

# Train/Valid Split
df_train, df_valid = train_test_split(
    dtf_mlm, test_size=0.15, random_state=SEED_SPLIT
)

len(df_train), len(df_valid)

# Convert to Dataset object
train_dataset = Dataset.from_pandas(df_train[['text']].dropna())
valid_dataset = Dataset.from_pandas(df_valid[['text']].dropna())

In [5]:
# '''
# bert-base-uncased  # 12-layer, 768-hidden, 12-heads, 109M parameters
# distilbert-base-uncased  # 6-layer, 768-hidden, 12-heads, 65M parameters
# '''

# MODEL = 'bert'
# bert_type = 'bert-base-uncased'

# if MODEL == 'distilbert':
#     TokenizerClass = DistilBertTokenizer
#     ModelClass = DistilBertForMaskedLM
# elif MODEL == 'bert':
#     TokenizerClass = BertTokenizer
#     ModelClass = BertForMaskedLM
# elif MODEL == 'roberta':
#     TokenizerClass = RobertaTokenizer
#     ModelClass = RobertaForMaskedLM
# elif MODEL == 'scibert':
#     TokenizerClass = AutoTokenizer
#     ModelClass = AutoModelForMaskedLM


# tokenizer = TokenizerClass.from_pretrained(
#             bert_type, use_fast=True, do_lower_case=False, max_len=MAX_SEQ_LEN
#             )
# model = ModelClass.from_pretrained(bert_type)

tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-uncased")
model = AutoModelForMaskedLM.from_pretrained("bert-base-multilingual-uncased")

Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [6]:
def tokenize_function(row):
    return tokenizer(
        row['text'],
        padding='max_length',
        truncation=True,
        max_length=MAX_SEQ_LEN,
        return_special_tokens_mask=True)

column_names = train_dataset.column_names

train_dataset = train_dataset.map(
    tokenize_function,
    batched=True,
    num_proc=multiprocessing.cpu_count(),
    remove_columns=column_names,
)

valid_dataset = valid_dataset.map(
    tokenize_function,
    batched=True,
    num_proc=multiprocessing.cpu_count(),
    remove_columns=column_names,
)

Map (num_proc=2):   0%|          | 0/79656 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/14058 [00:00<?, ? examples/s]

In [7]:
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)


steps_per_epoch = int(len(train_dataset) / TRAIN_BATCH_SIZE)

training_args = TrainingArguments(
    output_dir='./output',
    logging_dir='./log',
    num_train_epochs=5,
    do_train=True,
    do_eval=True,
    per_device_train_batch_size=TRAIN_BATCH_SIZE,
    per_device_eval_batch_size=EVAL_BATCH_SIZE,
    warmup_steps=LR_WARMUP_STEPS,
    save_steps=steps_per_epoch,
    save_total_limit=3,
    weight_decay=WEIGHT_DECAY,
    learning_rate=LEARNING_RATE,
    evaluation_strategy='epoch',
    save_strategy='epoch',
    load_best_model_at_end=True,
    metric_for_best_model='loss',
    greater_is_better=False,
    seed=SEED_TRAIN
)

# trainer = Trainer(
#     model=model,
#     args=training_args,
#     data_collator=data_collator,
#     train_dataset=train_dataset,
#     eval_dataset=valid_dataset,
#     tokenizer=tokenizer,
# )

In [8]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import numpy as np

def compute_metrics(p):
    pred, labels = p
    pred = np.argmax(pred, axis=1)
    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred, average='macro')
    precision = precision_score(y_true=labels, y_pred=pred, average='macro')
    f1 = f1_score(y_true=labels, y_pred=pred, average='macro')
    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

In [9]:
# tokenizer = AutoTokenizer.from_pretrained('sagorsarker/bangla-bert-base', use_fast = False, do_lower_case=True)
# model = AutoModelForMaskedLM.from_pretrained('sagorsarker/bangla-bert-base')

trainer = Trainer(
  model=model,
  data_collator=data_collator,
  train_dataset=train_dataset,
  eval_dataset=valid_dataset,
  # compute_metrics=compute_metrics,
  tokenizer=tokenizer,
  )



In [26]:
!rm -r tmp_trainer

In [27]:
!zip -r checkpoint.zip /content/checkpoint/checkpoint-12500

  adding: content/checkpoint/checkpoint-12500/ (stored 0%)
  adding: content/checkpoint/checkpoint-12500/generation_config.json (deflated 8%)
  adding: content/checkpoint/checkpoint-12500/tokenizer_config.json (deflated 43%)
  adding: content/checkpoint/checkpoint-12500/special_tokens_map.json (deflated 42%)
  adding: content/checkpoint/checkpoint-12500/training_args.bin (deflated 49%)
  adding: content/checkpoint/checkpoint-12500/vocab.txt (deflated 48%)
  adding: content/checkpoint/checkpoint-12500/config.json (deflated 53%)
  adding: content/checkpoint/checkpoint-12500/pytorch_model.bin (deflated 7%)
  adding: content/checkpoint/checkpoint-12500/trainer_state.json (deflated 77%)
  adding: content/checkpoint/checkpoint-12500/tokenizer.json (deflated 69%)
  adding: content/checkpoint/checkpoint-12500/optimizer.pt (deflated 7%)
  adding: content/checkpoint/checkpoint-12500/rng_state.pth (deflated 28%)
  adding: content/checkpoint/checkpoint-12500/scheduler.pt (deflated 48%)


In [11]:
trainer.train()


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
500,2.193100
1000,2.090300
1500,2.021900
2000,1.953300
2500,1.924200
3000,1.881600
3500,1.872300
4000,1.872200
4500,1.835900
5000,1.808300


RuntimeError: ignored

In [ ]:
# from transformers import AutoTokenizer, AutoModel

# # Replace these paths with the actual paths to your files
# tokenizer_config_path = "/content/tmp_trainer/checkpoint-7500/tokenizer.json"
# model_config_path = "/content/tmp_trainer/checkpoint-7500/config.json"
# model_checkpoint_path = "/content/tmp_trainer/checkpoint-7500/pytorch_model.bin"

# # Load the tokenizer
# # tokenizer = AutoTokenizer.from_json(tokenizer_config_path)

# # Load the model configuration
# model_config = AutoConfig.from_pretrained(model_config_path)

# # Load the model using the configuration
# model = AutoModel.from_pretrained(model_checkpoint_path, config=model_config)



Some weights of BertModel were not initialized from the model checkpoint at /content/tmp_trainer/checkpoint-7500/pytorch_model.bin and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(105879, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
 

OSError: ignored

In [28]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp /content/checkpoint.zip /content/drive/MyDrive/EMNLP

In [18]:
!zip /content/model.zip /content/tmp_trainer/checkpoint-12500/pytorch_model.bin

  adding: content/tmp_trainer/checkpoint-12500/pytorch_model.bin
zip I/O error: No space left on device
zip error: Output file write failure (write error on zip file)


In [16]:
!rm -r /content/tmp_trainer/checkpoint-9500

In [29]:
!cp /content/checkpoint.zip /content/drive/MyDrive/